In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# A typical training procedure for nn follows;
# - Define the nn has weigths
# - Iterate over a datasets of inputs
# - Process network through the network (forwardpass)
# - Compute loss (how far is  correct)
# - Propagate gradients into the learnable params
# - Update weigths

In [3]:
# Define Network
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)
        
    def forward(self, x):
        # max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [15]:
# we can access the network parameters via .parameters().
a = list(net.parameters())[0]

In [16]:
a.shape # 6 5 by 5 kernels sits here

torch.Size([6, 1, 5, 5])

In [19]:
input = torch.rand(1, 1, 32, 32) # why 4 dim array? (1,32,32) is image OK but other dim?
out = net(input)
out

tensor([[-0.0605, -0.0185, -0.0064,  0.0468,  0.0559,  0.1441,  0.0572,  0.0733,
         -0.0576,  0.0412]], grad_fn=<AddmmBackward>)

In [20]:
# zero the gradient buffer of all paramaters and backprop with random gradient
net.zero_grad() # documentation claims free used memory, so it's cool to call for start?
out.backward(torch.randn(1, 10)) # why backward accept a random array?

In [26]:
# - torch.nn is only supports mini-batches of samples NOT single sample so inputs other dimension is
# mini-batch count. If the nn  accept image we'll pass (1, 32, 32) sized tensor in this case we pass
# (1, 1, 32, 32) => (minibatch, im_channel, im_height, im_width)

# - doc gives this line as example;
# "For example, nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width."

# - input.unsqueeze(0) method can use for given tensor to add fake mini batch. See example below.

dummy_input = torch.rand(1,32,32)
print(dummy_input.shape)
unsqueezed_dummy_input = dummy_input.unsqueeze(0)
print(unsqueezed_dummy_input.shape)

torch.Size([1, 32, 32])
torch.Size([1, 1, 32, 32])


In [35]:
# Loss function
# there're multiple loss function in this link https://pytorch.org/docs/stable/nn.html

output = net(input) # nn's outputs
target = torch.rand(10) # create dummy target
target = target.unsqueeze(0) # bring dummy target same dim with nn's output
# target = target.view(1, -1) # bring dummy target same dim with nn's output
criterion = nn.MSELoss() # create  criterion obj
loss = criterion(output, target) # calc loss

print(loss.grad_fn.next_functions[0][0])


In [48]:
# backprop
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved variables after calling backward.

In [47]:
# update weights
# gradient descent weight = weight - learning_rate * gradient
# we can update weights
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)
    
# but torch includes multiple optimization algoritm in torch.optim

import torch.optim as optim
# create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.step()

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update